<a href="https://colab.research.google.com/github/meti-94/TextClassification/blob/main/BERT%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install necessary libraries 

In [22]:
# Install main transformers library
!pip install transformers==4.1.1

# Import libraries

In [23]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from transformers import AutoConfig, AutoTokenizer, AutoModel, BertForSequenceClassification
from sklearn.metrics import classification_report

# load up data to classifiy

In [24]:
## loading data
## X --> Texts
## y --> Labels

df = pd.read_excel('/content/drive/MyDrive/without_tabnak_train.xlsx')
X = df['text'].to_list(); y = df['topic'].to_list()

## Creating Test dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(X, 
                                                                      y, 
                                                                      random_state=42, 
                                                                      stratify=y, 
                                                                      test_size=.25)
## Create Valid dataset
train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, 
                                                                      train_labels, 
                                                                      random_state=42, 
                                                                      stratify=train_labels, 
                                                                      test_size=.1)


# Loading BERT model and tokenizer model

In [25]:
# v1.0
config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = BertForSequenceClassification.from_pretrained("HooshvareLab/bert-base-parsbert-uncased", num_labels=8)


Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

# Creating dataset and dataloader for X-y

In [26]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=200)
val_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=200)

In [27]:
# convert raw text file to proper dataset object (based on task)
import torch

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        # initialization
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # slicing method X[index]
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ClassificationDataset(train_encodings, train_labels)
valid_dataset = ClassificationDataset(val_encodings, valid_labels)

In [28]:
# transformers API for train :)
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=36,  # batch size per device during training
    per_device_eval_batch_size=36,   # batch size for evaluation
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    # weight_decay=0.01,               # strength of weight decay
    learning_rate= 5e-6,
    adam_epsilon = 1e-8, 
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    do_eval=True,
    evaluation_strategy = 'epoch'
    
    
)
# training_args.evaluation_strategy = EvaluationStrategy.EPOCH
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset

)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.400364,0.385962


KeyboardInterrupt: ignored

In [ ]:
# reading Test Data
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=50)
test_dataset = IMDbDataset(test_encodings, test_labels)

# Predicting on test set

In [ ]:
pred = trainer.predict(test_dataset)

# Evaluation

In [ ]:
print(classification_report(pred.label_ids, pred.predictions.argmax(-1)))

              precision    recall  f1-score   support

           0       0.94      0.82      0.88      1196
           1       0.55      0.81      0.65       320

    accuracy                           0.82      1516
   macro avg       0.74      0.81      0.76      1516
weighted avg       0.86      0.82      0.83      1516



In [ ]:
trainer.save_model("./classifier")

In [ ]:
cp -r classifier/ drive/MyDrive/